### SAM For LASSO

In [10]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, accuracy_score
import itertools, joblib, warnings, os, sys
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")

# ─────────────────────────────────────────────────────────────
# 1. Helper functions (define *before* run_once for joblib)
# ─────────────────────────────────────────────────────────────
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def soft_thresh(w, tau):
    return np.sign(w) * np.maximum(np.abs(w) - tau, 0.)

def pg_lasso(Xtr, ytr, lam, lr=0.1, n_iter=120):
    n, p = Xtr.shape
    w = np.zeros(p)
    for _ in range(n_iter):
        grad = Xtr.T @ (sigmoid(Xtr @ w) - ytr) / n
        w -= lr * grad
        w = soft_thresh(w, lr * lam)
    return w

def sam_pg_lasso(Xtr, ytr, lam, lr=0.1, rho=0.05, n_iter=120):
    n, p = Xtr.shape
    w = np.zeros(p)
    for _ in range(n_iter):
        grad = Xtr.T @ (sigmoid(Xtr @ w) - ytr) / n
        gnorm = np.linalg.norm(grad)
        w_adv = w + rho * grad / gnorm if gnorm > 0 else w
        grad_adv = Xtr.T @ (sigmoid(Xtr @ w_adv) - ytr) / n
        w -= lr * grad_adv
        w = soft_thresh(w, lr * lam)
    return w

# ─────────────────────────────────────────────────────────────
# 2.  One replicate runner (needs helpers already defined)
# ─────────────────────────────────────────────────────────────
def run_once(params, seed, lam=0.05, rho=0.05):
    n, p, flip_y, corr = params
    sep = 1.0 if corr == "low" else 0.7
    X, y = make_classification(
        n_samples=n, n_features=p,
        n_informative=int(0.2 * p),
        n_redundant=int(0.2 * p),
        flip_y=flip_y, class_sep=sep,
        random_state=seed,
    )
    X = StandardScaler().fit_transform(X)

    Xtr, Xtmp, ytr, ytmp = train_test_split(
        X, y, test_size=0.4, stratify=y, random_state=seed
    )
    Xval, Xte, yval, yte = train_test_split(
        Xtmp, ytmp, test_size=0.5, stratify=ytmp, random_state=seed
    )

    w0 = pg_lasso(Xtr, ytr, lam)
    w1 = sam_pg_lasso(Xtr, ytr, lam, rho=rho)

    loss0 = log_loss(yte, sigmoid(Xte @ w0))
    loss1 = log_loss(yte, sigmoid(Xte @ w1))
    acc0  = accuracy_score(yte, (sigmoid(Xte @ w0) > 0.5))
    acc1  = accuracy_score(yte, (sigmoid(Xte @ w1) > 0.5))

    return dict(
        n=n, p=p, flip_y=flip_y, corr=corr,
        seed=seed,
        loss_base=loss0, loss_sam=loss1,
        acc_base=acc0,  acc_sam=acc1,
    )

# ─────────────────────────────────────────────────────────────
# 3.  Simulation grid & parallel execution
# ─────────────────────────────────────────────────────────────
grid = {
    "n":      [4000, 20000],
    "p":      [200, 500],
    "flip_y": [0.05, 0.25],
    "corr":   ["low", "high"],
}
scenarios = list(itertools.product(*grid.values()))
N_REPS = 20   # adjust as desired

jobs = [(sc, 100 + r) for sc in scenarios for r in range(N_REPS)]

# Use threading backend to avoid pickling issues
results = joblib.Parallel(n_jobs=-1, backend="threading")(
    joblib.delayed(run_once)(*job) for job in tqdm(jobs, desc="Sim runs")
)

df = pd.DataFrame(results)

# ─────────────────────────────────────────────────────────────
# 4.  Aggregate & paired t-test
# ─────────────────────────────────────────────────────────────
from scipy.stats import ttest_rel

rows = []
for (n, p, flip, corr), grp in df.groupby(["n", "p", "flip_y", "corr"]):

    t_loss, p_val = ttest_rel(grp["loss_base"], grp["loss_sam"])

    rows.append(dict(
        n=n, p=p, flip_y=flip, corr=corr,
        # ▶ keep the raw means …
        loss_base = grp["loss_base"].mean(),
        loss_sam  = grp["loss_sam"].mean(),
        acc_base  = grp["acc_base"].mean(),
        acc_sam   = grp["acc_sam"].mean(),
        # ▶ … and compute deltas for quick scanning
        delta_loss = grp["loss_sam"].mean() - grp["loss_base"].mean(),
        delta_acc  = grp["acc_sam"].mean()  - grp["acc_base"].mean(),
        reps = len(grp),
        p_loss = p_val
    ))

summary = pd.DataFrame(rows)
print(summary.to_string(index=False, float_format="%.4f"))

Sim runs:   0%|          | 0/320 [00:00<?, ?it/s]

    n   p  flip_y corr  loss_base  loss_sam  acc_base  acc_sam  delta_loss  delta_acc  reps  p_loss
 4000 200  0.0500 high     0.6248    0.6169    0.6883   0.6936     -0.0078     0.0054    20  0.0000
 4000 200  0.0500  low     0.5392    0.5291    0.7727   0.7772     -0.0102     0.0045    20  0.0000
 4000 200  0.2500 high     0.6642    0.6583    0.6275   0.6328     -0.0059     0.0053    20  0.0000
 4000 200  0.2500  low     0.6185    0.6108    0.6919   0.6961     -0.0077     0.0041    20  0.0000
 4000 500  0.0500 high     0.6670    0.6597    0.6250   0.6380     -0.0073     0.0130    20  0.0000
 4000 500  0.0500  low     0.5996    0.5880    0.7257   0.7339     -0.0116     0.0083    20  0.0000
 4000 500  0.2500 high     0.6846    0.6800    0.5788   0.5924     -0.0046     0.0136    20  0.0000
 4000 500  0.2500  low     0.6534    0.6445    0.6556   0.6658     -0.0089     0.0102    20  0.0000
20000 200  0.0500 high     0.6285    0.6204    0.6834   0.6889     -0.0081     0.0055    20  0.0000


### LASSO

In [9]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import itertools, joblib, warnings
from tqdm.auto import tqdm
warnings.filterwarnings("ignore")

# ---------- helper functions ----------
def soft_thresh(w, t):
    return np.sign(w) * np.maximum(np.abs(w) - t, 0.)

def pg_lasso_lr(X, y, lam, lr=0.01, n_iter=200):
    n, p = X.shape
    w = np.zeros(p)
    for _ in range(n_iter):
        grad = X.T @ (X @ w - y) / n
        w -= lr * grad
        w = soft_thresh(w, lr * lam)
    return w

def sam_pg_lasso_lr(X, y, lam, lr=0.01, rho=0.05, n_iter=200):
    n, p = X.shape
    w = np.zeros(p)
    for _ in range(n_iter):
        grad = X.T @ (X @ w - y) / n
        norm = np.linalg.norm(grad)
        w_adv = w + rho * grad / norm if norm > 0 else w
        grad_adv = X.T @ (X @ w_adv - y) / n
        w -= lr * grad_adv
        w = soft_thresh(w, lr * lam)
    return w

# ---------- simulation grid ----------
grid = {
    "n":      [4000, 20000],
    "p":      [200, 500],
    "noise":  [5.0, 20.0],      # low vs high noise std
    "corr":   ["low", "high"],  # correlation via effective_features
}
scenarios = list(itertools.product(*grid.values()))
N_REPS = 10

def run_once(params, seed, lam=0.1, rho=0.05):
    n, p, noise_std, corr = params
    eff = 0.3 if corr == "low" else 0.1  # effective sparsity ratio

    X, y, coef = make_regression(
        n_samples=n, n_features=p, n_informative=int(p*eff),
        noise=noise_std, coef=True, random_state=seed
    )
    # add feature correlation if high
    if corr == "high":
        # correlate first 50 features
        cov = 0.8
        X[:, :50] += cov * X[:, 50:100] if p >= 100 else 0

    X = StandardScaler().fit_transform(X)

    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.4,
                                              random_state=seed)
    w0 = pg_lasso_lr(X_tr, y_tr, lam)
    w1 = sam_pg_lasso_lr(X_tr, y_tr, lam, rho=rho)

    mse0 = mean_squared_error(y_te, X_te @ w0)
    mse1 = mean_squared_error(y_te, X_te @ w1)
    nz0 = np.count_nonzero(w0)
    nz1 = np.count_nonzero(w1)

    return dict(n=n, p=p, noise=noise_std, corr=corr,
                mse_base=mse0, mse_sam=mse1,
                nz_base=nz0, nz_sam=nz1, seed=seed)

jobs = [(sc, 123 + rep) for sc in scenarios for rep in range(N_REPS)]

results = joblib.Parallel(n_jobs=-1, backend="threading")(
    joblib.delayed(run_once)(*job) for job in tqdm(jobs, desc="Lasso sims")
)

df = pd.DataFrame(results)

summary = (df.groupby(["n","p","noise","corr"])
             .agg(MSE_base=("mse_base","mean"),
                  MSE_sam =("mse_sam","mean"),
                  ΔMSE     =("mse_sam", lambda x:
                              x.mean()-df.loc[x.index,"mse_base"].mean()),
                  NZ_base =("nz_base","mean"),
                  NZ_sam  =("nz_sam","mean"))
             .reset_index())

#"SAM vs baseline Lasso (linear regression)"
summary.round(4)

Lasso sims:   0%|          | 0/160 [00:00<?, ?it/s]

,n,p,noise,corr,MSE_base,MSE_sam,ΔMSE,NZ_base,NZ_sam
0,4000,200,5.0,high,4720.6846,4717.7590,-2.9255,197.9,197.9
1,4000,200,5.0,low,6592.8101,6587.0606,-5.7495,199.0,199.0
2,4000,200,20.0,high,5100.7024,5097.7989,-2.9035,198.8,198.8
3,4000,200,20.0,low,6981.7081,6975.9701,-5.7380,198.8,198.8
4,4000,500,5.0,high,11790.8682,11786.1766,-4.6916,498.2,498.2
5,4000,500,5.0,low,27441.2568,27432.1028,-9.1540,499.2,499.4
6,4000,500,20.0,high,12189.1378,12184.4683,-4.6695,499.1,499.2
7,4000,500,20.0,low,27812.4825,27803.3362,-9.1462,499.1,499.2
8,20000,200,5.0,high,4568.8837,4566.0600,-2.8237,193.4,193.4
9,20000,200,5.0,low,4086.7925,4081.5247,-5.2678,193.9,194.0


### Quantile